In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load data using Data Genrator

In [ ]:
import tensorflow as tf
from tensorflow import keras
print("TensorFlow version is ", tf.__version__)
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from sklearn.metrics import classification_report
import numpy as np
from sklearn.utils import shuffle
from keras import regularizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
import keras.layers as Layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.optimizers as Optimizer
print(tf.__version__)
from keras import applications
from tensorflow import keras

# stacked generalization with linear meta model on blobs dataset
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from keras.utils import to_categorical
from numpy import dstack

# Use_PreTrain_model()

In [ ]:
def use_PreTrain_model(): 
    import tensorflow as tf
    img_height,img_width = 224,224 
    IMG_SHAPE = (img_height,img_width, 3)
    inp = tf.keras.Input(shape=(img_height,img_width, 3))
    inp2 = tf.keras.layers.Concatenate()([inp, inp, inp])

    #base_model = tf.compat.v2.keras.applications.ResNet152(include_top=False, weights='imagenet', input_tensor=None, input_shape=IMG_SHAPE, pooling=None, classes=2)

    #base_model = tf.compat.v2.keras.applications.ResNet152(include_top=False, weights='imagenet', input_tensor=None, input_shape=IMG_SHAPE, pooling=None, classes=2)

    #base_model = tf.compat.v2.keras.applications.VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=IMG_SHAPE, pooling=None, classes=3)
    #base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_tensor=None, input_shape=IMG_SHAPE, pooling=None, classes=3)
    #base_model =  tf.compat.v2.keras.applications.Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=IMG_SHAPE, pooling=None, classes=1000)
    #base_model = tf.compat.v2.keras.applications.VGG19(include_top=False, weights='imagenet', input_tensor=None, input_shape=IMG_SHAPE, pooling=None, classes=1000)
    #base_model = tf.keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=IMG_SHAPE, pooling=None, classes=1000)
    base_model =tf.keras.applications.DenseNet121(include_top=False, weights='imagenet', input_tensor=None, input_shape=IMG_SHAPE, pooling=None, classes=1000)
    #base_model =tf.keras.applications.DenseNet169(include_top=False, weights='imagenet', input_tensor=None, input_shape=IMG_SHAPE, pooling=None, classes=1000)
    #base_model = tf.keras.applications.MobileNetV2(include_top=False, weights='imagenet', input_tensor=None, input_shape=IMG_SHAPE, pooling=None, classes=1000)
    #base_model =tf.keras.applications.NASNetLarge(include_top=False, weights='imagenet', input_tensor=None, input_shape=IMG_SHAPE, pooling=None, classes=1000)
  
    
    
    base_model.trainable = True
    # Let's take a look to see how many layers are in the base model
    print("Number of layers in the base model: ", len(base_model.layers))
    # Fine-tune from this layer onwards
    fine_tune_at =422

    # Freeze all the layers before the `fine_tune_at` layer
    for layer in base_model.layers[:fine_tune_at]:
      layer.trainable =  False
         
    model = tf.keras.Sequential([
          base_model,
          keras.layers.Conv2D(512, (3, 3), activation='relu'),
          keras.layers.Conv2D(512, (3, 3), activation='relu'),

          keras.layers.GlobalAveragePooling2D(),
          #MCDropout(rate=0.5),
          keras.layers.Dense(3, activation='softmax')
    ])
    return model 
#model.summary()

In [ ]:
def dataGen(batch, path) :
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    # All images will be rescaled by 1./255
    train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=15,  
                                       width_shift_range=0.4,height_shift_range=0.3, horizontal_flip=True,
                                       shear_range=0.2, zoom_range=0.2)
    # fit the data augmentation
    #train_datagen.fit(x_train)
      
    valid_datagen = ImageDataGenerator(rescale=1./255)#,featurewise_center=True, 
                                       #featurewise_std_normalization=True, rotation_range=20,   
                                       #width_shift_range=0.2,height_shift_range=0.2, horizontal_flip=True)

    test_datagen = ImageDataGenerator(rescale=1./255)
    size=(224,224)
        
    train_generator = train_datagen.flow_from_directory(
            path+'Train',  # This is the source directory for training images
            target_size=size,  # All images will be resized to 150x150
            batch_size=batch,
            # Since we use binary_crossentropy loss, we need binary labels
            class_mode='categorical')
    label_map = (train_generator.class_indices)

    test_generator = test_datagen.flow_from_directory(
            path+'test/',  # This is the source directory for training images
            target_size=size,  # All images will be resized to 150x150
            batch_size=batch,
            # Since we use binary_crossentropy loss, we need binary labels
            class_mode='categorical',
            shuffle=False)

    valid_generator = valid_datagen.flow_from_directory(
            path+'validation/',  # This is the source directory for training images
            target_size=size,  # All images will be resized to 150x150
            batch_size=batch,
            # Since we use binary_crossentropy loss, we need binary labels
            class_mode='categorical')
    return train_generator, test_generator, valid_generator


In [ ]:
#from keras import optimizers
#model = get_model(mc=True)
model = use_PreTrain_model()
path='../input/covid19cxr2-dataset/fold4/fold4/'

train_generator, test_generator, valid_generator = dataGen(16,path)
#model=use_PreTrain_model()
n_epochs =10

model.compile(tf.keras.optimizers.SGD(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])# define learning rate callback
#model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001, decay=1e-6),
#              loss='binary_crossentropy',metrics=['Accuracy'])# define learning rate callback

checkpointer = ModelCheckpoint('mob_model-1.h5',monitor='val_loss', verbose=1, save_best_only=True)

history = model.fit_generator(train_generator, epochs=n_epochs, verbose= 1, shuffle = True, 
                              validation_data = valid_generator,steps_per_epoch=40,
                              validation_steps=2, callbacks=[checkpointer])

model.save('mob_model.h5')

# evaluate the model
_, train_acc = model.evaluate(train_generator, verbose=0)
_, test_acc = model.evaluate(test_generator, verbose=0)

print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
model1 = load_model('mob_model-1.h5')
_, test_acc = model1.evaluate(test_generator, verbose=0)
print('Train by mob_model-1: %.3f' % (test_acc))


fold1-608,  fold2-,   fold3-,  fold4-,  fold5-

In [ ]:
path='../input/covid19cxr-dataset2/fold5/fold5/'
train_generator, test_generator, valid_generator = dataGen(16,path) 
test_generator.reset()
#test_Images, test_labels = test_generator.next()
#train_generator, test_generator, valid_generator = dataGen(304,path)
#validation_Images, validation_labels = valid_generator.next() 

In [ ]:
from sklearn.metrics import confusion_matrix
from imblearn.metrics import sensitivity_specificity_support
from sklearn.metrics import classification_report
from imblearn.metrics import geometric_mean_score
import seaborn as sn
import pandas as pd
model = load_model('../input/stack-cnn-submodels/vgg19/vgg19/vgg_F5_model_9.h5')
test_generator.reset()
pred = model.predict_generator(test_generator, verbose=0, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False)
#pred = model.predict_generator(test_generator, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)
#pred_label, true_label = test_Images, test_labels
# evaluate standalone models on test dataset
pred_label=np.argmax(pred,axis=1)
true_label = test_generator.classes

loss, acc = model.evaluate_generator(test_generator,steps=None)

target_names = ['COVID19',  'Normal','Pneumonia']
result = sensitivity_specificity_support(true_label, pred_label, average='macro')

print("Sensitivity: {:5.2f}%".format(100*result[0]), "specificity {:5.2f}%".format(100*result[1]), 
      "Accuracy: {:5.2f}%".format(100*acc),'\n')

report=classification_report(true_label, pred_label, target_names=target_names, digits=4)
print(report)

f = open( 'report.txt', 'w' )
f.write(report)
f.close()


disp = confusion_matrix(true_label, pred_label)
disp.astype('int')
pd.options.display.float_format = '{:.5f}'.format
df_cm = pd.DataFrame(disp, target_names, target_names)

fig, ax = plt.subplots(figsize=(4,3))
sn.set(font_scale=1.2) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 13},ax=ax, cmap="YlGnBu" , fmt='g',cbar=False)#, fontsize =12)
plt.ylabel('Actual',fontsize=16,fontweight='bold')
plt.xlabel('Predicted',fontsize=16,fontweight='bold')
plt.ioff()
import pylab
pylab.savefig('fold_conf_mat.eps', format = 'eps',bbox_inches='tight')

##### Sensivity and PPV ------------------------------------------------------------------------------------
matrix = confusion_matrix(true_label, pred_label)
matrix = matrix.astype('float')
#cm_norm = matrix / matrix.sum(axis=1)[:, np.newaxis]
#print(matrix)
#class_acc = np.array(cm_norm.diagonal())

Gmean = geometric_mean_score(true_label, pred_label, average='multiclass')
print('Gmean: {:.4f}'.format(Gmean))

class_acc = [matrix[i,i]/np.sum(matrix[i,:]) if np.sum(matrix[i,:]) else 0 for i in range(len(matrix))]
print('Sens COVID-19: {0:.3f}, Normal: {1:.3f}, Pneumonia: {2:.3f}'.format(class_acc[0], class_acc[1], class_acc[2]))
                                                                             
ppvs = [matrix[i,i]/np.sum(matrix[:,i]) if np.sum(matrix[:,i]) else 0 for i in range(len(matrix))]
print('PPV COVID-19: {0:.3f}, Normal: {1:.3f}, Pneumonia: {2:.3f}'.format(ppvs[0],  ppvs[1], ppvs[2]))


print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score
from keras.utils import to_categorical

true_label2 = to_categorical(true_label)
#pred_label = yscore
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
yscore=pred
roc_auc = dict()
n_classes = 3
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(true_label2[:, i], yscore[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(true_label2.ravel(), yscore.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# First aggregate all false positive rates
lw = 2
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

#fpr["macro"] = all_fpr
#tpr["macro"] = mean_tpr
#roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='Micro-Avg(AUC={0:0.3f})'
               ''.format(roc_auc["micro"]),
         color='k', linestyle=':', linewidth=1.2)



plt.plot(fpr[0], tpr[0], color='b',
         lw=lw, label='COVID-19(AUC=%0.3f)' % roc_auc[0],linewidth=1.1)

plt.plot(fpr[1], tpr[1], color='g',
         lw=lw, label='Normal(AUC=%0.3f)' % roc_auc[1],linewidth=1.2)

plt.plot(fpr[2], tpr[2], color='r',
         lw=lw, label='Pneumonia(AUC=%0.3f)' % roc_auc[2],linewidth=1)


import matplotlib 

size=12
params = {'legend.fontsize': 8.5,
          'figure.figsize': (3,2),
          'axes.labelsize': size,
          'axes.titlesize': size,
          'xtick.labelsize': size*0.8,
          'ytick.labelsize': size*0.8,
          'lines.linewidth':1,
          'axes.titlepad': 6}

plt.rcParams.update(params)

#plt.style.use('classic')
plt.rc('lines', linewidth=4)
#plt.legend.fontsize = 10
#plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([-0.007, 1.0])
plt.ylim(top=1.003)
plt.ylim(bottom=.70)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
#plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
#plt.rcParams["figure.figsize"] = (6,4)
pylab.savefig('foldROC.eps', format = 'eps',bbox_inches='tight')
plt.show()


# Grad-CAM


In [ ]:
from keras.models import *
from keras.callbacks import *
import keras.backend as K
#from model import *
#from data import *
import cv2
import argparse
# import Keras's functional api
from keras.models import Model

model = load_model('../input/models-covid19xray2/fold1/EfN_model.h5')
model.summary()

In [ ]:


# Load an color image in grayscale
img1 = cv2.imread("../input/covid19cxr/fold1/fold1/Train/Train/COVID19/COVID-00058.jpg")
img1 =cv2.resize(img1, (224, 224))
img2 = cv2.imread("../input/covid19cxr/fold1/fold1/Train/Train/COVID19/COVID-00060-b.jpg")
img2 =cv2.resize(img2, (224, 224))
img3 = cv2.imread("../input/covid19cxr/fold1/fold1/Train/Train/COVID19/COVID-00077.jpg")
img3 =cv2.resize(img3, (224, 224))


In [ ]:
img_path ="../input/covid19cxr/fold1/fold1/Train/Train/PNEUMONIA/PNEUMONIA_person14_virus_44.jpeg"
original_img = cv2.imread(img_path, 1)
original_img =cv2.resize(original_img, (224, 224))
width, height = 224, 224

original_img  = original_img/255
original_img  = original_img.astype('float')
original_img.shape

#Reshape to the network input shape (3, w, h).
img = np.array([np.transpose(np.float32(original_img), (0, 1, 2))])

In [ ]:
def show_img(img):
    img  = np.array(img,dtype='float')
    #img = img.reshape((224,224))
     
    plt.imshow(img)
#x = np.random.randint(0,X_test.shape[0])
show_img(original_img)

In [ ]:
#Get the 512 input weights to the softmax.
class_weights = model.layers[-1].get_weights()[0]
#final_conv_layer = "swish_78"
get_output = K.function([model.layers[0].input], [model.get_layer("conv2d_104").output, model.layers[-1].output])
[conv_outputs, predictions] = get_output([img])
conv_outputs = conv_outputs[0, :, :, :]

In [ ]:
np.shape(w)

In [ ]:
#Create the class activation map.
cam = np.zeros(dtype = np.float32, shape = conv_outputs.shape[1:3])
for i, w in enumerate(class_weights[1 ,:]): 
    cam += w * conv_outputs[i, :, :]
print("predictions", predictions)
cam /= np.max(cam)
cam = cv2.resize(cam, (height, width))
heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
heatmap[np.where(cam < 0.2)] = 0
img = heatmap*0.5 + original_img
#cv2.imwrite('../input/output/',img)

In [ ]:
def show_img(img):
    img  = np.array(img,dtype='float')
    #img = img.reshape((224,224))
     
    plt.imshow(img)
#x = np.random.randint(0,X_test.shape[0])
show_img(img)

In [ ]:
from skimage.io import imread, imshow, imread_collection, concatenate_images
from matplotlib import pyplot as plt
%matplotlib inline

imshow(heatmap.astype(np.uint8))
plt.show()

In [ ]:
cv2.imshow('img',img) 
#np.shape(img)

In [ ]:
def visualize_class_activation_map(model_path, img_path, output_path):
        model = load_model(model_path)
        original_img = cv2.imread(img_path, 1)
        width, height, _ = original_img.shape

        #Reshape to the network input shape (3, w, h).
        img = np.array([np.transpose(np.float32(original_img), (2, 0, 1))])
        
        #Get the 512 input weights to the softmax.
        class_weights = model.layers[-1].get_weights()[0]
        final_conv_layer = get_output_layer(model, "conv5_3")
        get_output = K.function([model.layers[0].input], [final_conv_layer.output, model.layers[-1].output])
        [conv_outputs, predictions] = get_output([img])
        conv_outputs = conv_outputs[0, :, :, :]

        #Create the class activation map.
        cam = np.zeros(dtype = np.float32, shape = conv_outputs.shape[1:3])
        for i, w in enumerate(class_weights[:, 1]):
                cam += w * conv_outputs[i, :, :]
        print "predictions", predictions
        cam /= np.max(cam)
        cam = cv2.resize(cam, (height, width))
        heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
        heatmap[np.where(cam < 0.2)] = 0
        img = heatmap*0.5 + original_img
        cv2.imwrite(output_path, img)


In [ ]:
import cv2

In [ ]:
Y_train = np.zeros((1, 224, 224, 3),dtype=np.uint8)
Y_train[0] = img1
Y_train[1] = img2
Y_train[2] = img3
                   
np.shape(Y_train)

In [ ]:
import pandas as pd
from keras.applications.vgg16 import decode_predictions
preds = model.predict(Y_train)
predictions = pd.DataFrame(decode_predictions(preds, top=3)[0],columns=['col1','category','probability']).iloc[:,1:]
print('PREDICTION:',predictions.loc[0,'category'])

In [ ]:
argmax = np.argmax(preds[0])
output = model.output[:, argmax]

In [ ]:
model._layers_by_depth

In [ ]:
last_conv_layer = model.get_layer('conv2d_104')

In [ ]:
grads = K.gradients(output, last_conv_layer.output)[0]

In [ ]:
pooled_grads = K.mean(grads, axis=(0, 1, 2))

In [ ]:
K.clear_session()

In [ ]:
iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])

In [ ]:
train_img_size_h,train_img_size_w =224,224

In [ ]:
# check the prediction for 10 test images
for idx in range(3):   
    # get the feature map of the test image
    features_for_one_img = features[idx, :, :, :]

    # map the feature map to the original size
    height_roomout = train_img_size_h / features_for_one_img.shape[0]
    width_roomout = train_img_size_w / features_for_one_img.shape[1]
    cam_features = sp.ndimage.zoom(features_for_one_img, (height_roomout, width_roomout, 1), order=2)
        
    # get the predicted label with the maximum probability
    pred = np.argmax(results[idx])
    
    # prepare the final display
    plt.figure(facecolor='white')
    
    # get the weights of class activation map
    cam_weights = gap_weights[:, pred]

    # create the class activation map
    cam_output = np.dot(cam_features, cam_weights)
    
    # draw the class activation map
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    
    buf = 'Predicted Class = ' + fashion_name[pred] + ', Probability = ' + str(results[idx][pred])
    plt.xlabel(buf)
    plt.imshow(t_pic[idx], alpha=0.5)
    plt.imshow(cam_output, cmap='jet', alpha=0.5)
     
    plt.show()  

In [ ]:
# Get normalized input. VGG network handles the normalized image internally. 
#batch1_img = img1.reshape((1, 224, 224, 3))
batch1_label = np.array([1 if i == 242 else 0 for i in range(1000)])  # 1-hot result for Boxer
batch1_label = batch1_label.reshape(1, -1)

#batch2_img = img2.reshape((1, 224, 224, 3))
batch2_label = np.array([1 if i == 155 else 0 for i in range(1000)])  # 1-hot result for Shih-Tzu
batch2_label = batch2_label.reshape(1, -1)

#batch3_img = img3.reshape((1, 224, 224, 3))
batch3_label = np.array([1 if i == 292 else 0 for i in range(1000)])  # 1-hot result for tiger
batch3_label = batch3_label.reshape(1, -1)

batch_img = np.concatenate((img1, img2, img3), 0)
batch_label = np.concatenate((batch1_label, batch2_label, batch3_label), 0)

batch_size = 3

# for i in range(batch_size):
#     print('See visualization of below category')
#     utils.print_prob(batch_label[i], './synset.txt')

# Create tensorflow graph for evaluation
eval_graph = tf.Graph()
with eval_graph.as_default():
    with eval_graph.gradient_override_map({'Relu': 'GuidedRelu'}):
    
        images = tf.placeholder("float", [batch_size, 224, 224, 3])
        labels = tf.placeholder(tf.float32, [batch_size, 1000])

        vgg = vgg16.Vgg16()
        
        vgg.build(images)
        cost = (-1) * tf.reduce_sum(tf.multiply(labels, tf.log(vgg.prob)), axis=1)
        print('cost:', cost)
        # cost = tf.reduce_sum((vgg.prob - labels) ** 2)
        
        
        # gradient for partial linearization. We only care about target visualization class. 
        y_c = tf.reduce_sum(tf.multiply(vgg.fc8, labels), axis=1)
        print('y_c:', y_c)
        # Get last convolutional layer gradient for generating gradCAM visualization
        target_conv_layer = vgg.pool5
        target_conv_layer_grad = tf.gradients(y_c, target_conv_layer)[0]

        # Guided backpropagtion back to input layer
        gb_grad = tf.gradients(cost, images)[0]

        init = tf.global_variables_initializer()


In [ ]:
# Run tensorflow 

with tf.Session(graph=eval_graph) as sess:    
    sess.run(init)
    
    prob = sess.run(vgg.prob, feed_dict={images: batch_img})
    
    gb_grad_value, target_conv_layer_value, target_conv_layer_grad_value = sess.run([gb_grad, target_conv_layer, target_conv_layer_grad], feed_dict={images: batch_img, labels: batch_label})
    
    
    for i in range(batch_size):
        utils.print_prob(prob[i], './synset.txt')
        # VGG16 use BGR internally, so we manually change BGR to RGB
        gradBGR = gb_grad_value[i]
        gradRGB = np.dstack((
            gradBGR[:, :, 2],
            gradBGR[:, :, 1],
            gradBGR[:, :, 0],
        ))
        utils.visualize(batch_img[i], target_conv_layer_value[i], target_conv_layer_grad_value[i], gradRGB)

In [ ]:
results = []
names = []

sensitivity = [94.03, 91.45, 92.54, 91.78, 94.17] 

results.append(sensitivity)
names.append('Sensitivity')
specificity = (96.99, 94.47, 95.15, 94.42, 95.41 ) 
results.append(specificity)
names.append('Specificity')
accuracy = (94.95, 90.53, 91.52, 89.86, 92.01) 
results.append(accuracy)
names.append('Accuracy')
PPV = (96.23, 89.83, 92.30, 86.20, 91.70)
results.append(PPV)
names.append('PPV')

In [ ]:
import pandas
import matplotlib.pyplot as plt
import statistics 
# boxplot algorithm comparison
results = []
names = []

#sensitivity = [94.03, 91.45, 92.54, 91.78, 94.17] 
with open("../input/models-covid19xray2/fold1/acc.txt", "rb") as fp:   # Unpickling
    accs = pickle.load(fp)
results.append(accs)
names.append('fold1')
with open("../input/models-covid19xray2/fold2/acc.txt", "rb") as fp:   # Unpickling
    accs = pickle.load(fp)
results.append(accs)
names.append('fold2')
with open("../input/models-covid19xray2/fold3/acc.txt", "rb") as fp:   # Unpickling
    accs = pickle.load(fp)
results.append(accs)
names.append('fold3')
with open("../input/models-covid19xray2/fold4/acc.txt", "rb") as fp:   # Unpickling
    accs = pickle.load(fp)
results.append(accs)
names.append('fold4')
with open("../input/models-covid19xray2/fold5/acc.txt", "rb") as fp:   # Unpickling
    accs = pickle.load(fp)
results.append(accs)
names.append('fold5')
#print("Standard Deviation of sensi {:.4f} ".format(statistics.stdev(accs))) 
#print("Variance of sample set is {:.5f}".format(statistics.variance(accs))) 

fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results, showfliers=True, widths =0.4)
ax.set_xticklabels(names)
plt.rcParams["figure.figsize"] = (8,4)
plt.savefig('boxplot')
plt.show()

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# Make some fake data.
a = b = np.arange(0, 3, .02)
c = np.exp(a)
d = c[::-1]

# Create plots with pre-defined labels.
fig, ax = plt.subplots()
ax.plot(a, c, 'k--', label='Model length')
ax.plot(a, d, 'k:', label='Data length')
ax.plot(a, c + d, 'k', label='Total message length')

legend = ax.legend(loc='upper center', shadow=True, fontsize='x-large')

# Put a nicer background color on the legend.
legend.get_frame().set_facecolor('#00FFCC')

plt.show()

In [ ]:
idx = 247
plt.imshow(test_Images[idx][:,:,0])
p0 = np.array([p[idx] for p in mc_predictions])
print("posterior mean: {}".format(p0.mean(axis=0).argmax()))
print("true label: {}".format(test_labels[idx].argmax()))
print()
# probability + variance
for i, (prob, var) in enumerate(zip(p0.mean(axis=0), p0.std(axis=0))):
    print("class: {}; proba: {:.1%}; var: {:.2%} ".format(i, prob, var))

x, y = list(range(len(p0.mean(axis=0)))), p0.mean(axis=0)
plt.plot(x, y);
fig, axes = plt.subplots(3, 1, figsize=(12,12))

for i, ax in enumerate(fig.get_axes()):
    ax.hist(p0[:,i], bins=100, range=(0,1));
    ax.set_title(f"class {i}")
    ax.label_outer()
    
m = tf.keras.metrics.Accuracy()
m.update_state(true_label, pred_label)
m.result().numpy()


In [ ]:
# -*- coding: utf-8 -*-
"""
Standard Normal Distribution
Author: Balamurali M
"""

import numpy as np
import matplotlib.pyplot as plt

class norm1:
    def __init__(self, a1, b1, c1):
        self.a1 = a1
        self.b1 = b1
        self.c1 = c1
        
    def dist_curve(self):
        plt.plot(self.c1, 1/(self.b1 * np.sqrt(2 * np.pi)) *
            np.exp( - (self.c1 - self.a1)**2 / (2 * self.b1**2) ), linewidth=2, color='y')
        plt.show()

#mean 0 and sd 1 for the standard normal distribution


mean, sd = sample_mean,pop_stdev


c = sample#np.random.normal(mean, sd, 3000)
        
w1, x1, z1 = plt.hist(c, 10, density=True,stacked=True,cumulative=False) #hist

hist1 = norm1(mean, sd, x1)
plot1 = hist1.dist_curve()